# sanitize

In [2]:
from evalplus.data import write_jsonl
import json
import os
mbpp_plus_task_ids = json.load(open('../redial/redial_gold/mbpp_plus_task_ids.json', 'r'))


In [3]:
def add_dummy_task_ids(eval_path,
                       mbpp_plus_task_ids):
    with open(eval_path, 'r') as f:
        to_eval = [json.loads(l) for l in f]
    to_eval_task_ids = set([problem['task_id'] for problem in to_eval])
    for task_id in mbpp_plus_task_ids:
        if task_id not in to_eval_task_ids:
            to_eval.append({'task_id':task_id, 'solution':''})

    # write back to the file
    write_jsonl(eval_path, to_eval)


def sanitize_files(alg_dir,
                   mbpp_plus_task_ids):
    par_dirs = os.listdir(alg_dir)
    for parent_directory in par_dirs:
        for child_dir in os.listdir(os.path.join(alg_dir, parent_directory)):
            child_dir_path = os.path.join(alg_dir, parent_directory, child_dir)
            if os.path.isdir(child_dir_path):
                print(f'Processing directory: {child_dir_path}')
                # Iterate through all files in the child directory
                for temp_dir in os.listdir(child_dir_path):
                    temp_dir_path = os.path.join(child_dir_path, temp_dir)
                    if os.path.isdir(temp_dir_path):
                        for file in os.listdir(temp_dir_path):
                            file_path = os.path.join(temp_dir_path, file)
                            # Check if the file ends with '_new.jsonl'
                            if os.path.isfile(file_path) and 'to_eval' in file and 'sanitized' not in file:
                                if file_path.replace('.jsonl', '-sanitized.jsonl') in os.listdir(temp_dir_path):
                                    continue
                                # phi output unparsed format is weird so skip it
                                if 'phi' in file_path and 'unparsed' in file_path:
                                    continue
                                # Execute the command
                                if 'mbpp' in file_path:
                                    # read jsonl
                                    with open(file_path, 'r') as f:
                                        if len([json.loads(l) for l in f]) != 150:
                                            continue
                                else:
                                    with open(file_path, 'r') as f:
                                        if len([json.loads(l) for l in f]) != 164:
                                            continue
                                # continue if the sanitized file already exists
                                if os.path.exists(file_path.replace('.jsonl', '-sanitized.jsonl')):
                                    continue
                                os.system(f'evalplus.sanitize --samples "{file_path}"')
                                if 'mbpp' in file_path:
                                    add_dummy_task_ids(file_path.replace('.jsonl', '-sanitized.jsonl'), mbpp_plus_task_ids)    

# test

In [5]:
import os
import subprocess
import concurrent.futures

def run_subprocess(file, child_dir_path):
    # try:
        # if os.path.isfile(file) and '-sanitized' in file:
    if 'humaneval' in file:
        data_name = 'humaneval'
    else:
        data_name = 'mbpp'

    unix_child_dir_path = os.path.abspath(child_dir_path).replace('\\', '/')
    result = subprocess.run(
        ['docker', 'run', '-v', f'{unix_child_dir_path}:/app', 'ganler/evalplus:latest', '--dataset', data_name, '--samples', file],
        capture_output=True,
        text=True
    )
    # Print the captured output
    print(result.stdout)
    if result.stderr:
        print(result.stderr)
def test_code(alg_dir,
              max_workers=10):
    par_dirs = os.listdir(alg_dir)
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for parent_directory in par_dirs:
            for child_dir in os.listdir(os.path.join(alg_dir, parent_directory)):
                child_dir_path = os.path.join(alg_dir, parent_directory, child_dir)
                if os.path.isdir(child_dir_path):
                    print(f'Processing directory: {child_dir_path}')
                    # Iterate through all files in the child directory
                    for temp_dir in os.listdir(child_dir_path):
                        file_path = os.path.join(child_dir_path, temp_dir)
                        if os.path.isdir(file_path):
                            for file in os.listdir(file_path):
                                f_path = os.path.join(file_path, file)
                                if os.path.isfile(f_path) and '-sanitized' in file and 'eval_results' not in file and file.replace('-sanitized.jsonl', '-sanitized_eval_results.json') not in os.listdir(file_path):
                                    future = executor.submit(run_subprocess, file, file_path)
                                    futures.append(future)
                                    # Wait for the subprocess to complete before clearing the Docker cache
                                    future.add_done_callback(lambda fut: executor.submit(subprocess.run, ['docker', 'system', 'prune', '-f'], capture_output=True, text=True))
        for future in concurrent.futures.as_completed(futures):
            future.result()  # To raise any exceptions that occurred

# sanitize and test

In [6]:
def sanitize_and_test(alg_dir, mbpp_plus_task_ids=mbpp_plus_task_ids, max_workers=10):
    sanitize_files(alg_dir, mbpp_plus_task_ids)
    test_code(alg_dir, max_workers=max_workers)

put your evaluation directory here

In [ ]:
sanitize_and_test('eval_dir')